In this notebook, we focus on solving the following problem:

QUESTION 1: Alice has some cards with numbers written on them. She arranges the cards in decreasing order, and lays them out face down in a sequence on a table. She challenges Bob to pick out the card containing a given number by turning over as few cards as possible. Write a function to help Bob locate the card.


This may seem like a simple problem, especially if you're familiar with the concept of binary search, but the strategy and technique we learning here will be widely applicable, and we'll soon use it to solve harder problems.

The first step is to state the problem clearly and precisely in abstract terms.


In this case, for instance, we can represent the sequence of cards as a list of numbers. Turning over a specific card is equivalent to accessing the value of the number at the corresponding position the list.


The problem can now be stated as follows:

Problem
We need to write a program to find the position of a given number in a list of numbers arranged in decreasing order. We also need to minimize the number of times we access elements from the list.

Input
cards: A list of numbers sorted in decreasing order. E.g. [13, 11, 10, 7, 4, 3, 1, 0]
query: A number, whose position in the array is to be determined. E.g. 7
Output
position: The position of query in the list cards. E.g. 3 in the above case (counting from 0)

In [2]:
def locate_card(cards, query):
    pass

Before we start implementing our function, it would be useful to come up with some example inputs and outputs which we can use later to test out problem. We'll refer to them as test cases.

Here's the test case described in the example above.

In [3]:
cards = [13, 11, 10, 7, 4, 3, 1, 0]
query = 7
output = 3

result = locate_card(cards, query)
print(result)

None


In [4]:
result == output

False

Obviously, the two result does not match the output as we have not yet implemented the function.

We'll represent our test cases as dictionaries to make it easier to test them once we write implement our function. For example, the above test case can be represented as follows:

In [5]:
test = {
    'input': { 
        'cards': [13, 11, 10, 7, 4, 3, 1, 0], 
        'query': 7
    },
    'output': 3
}

In [6]:
# The function can now be tested as follows.

locate_card(**test['input']) == test['output']

False

Our function should be able to handle any set of valid inputs we pass into it. Here's a list of some possible variations we might encounter:

The number query occurs somewhere in the middle of the list cards.
query is the first element in cards.
query is the last element in cards.
The list cards contains just one element, which is query.
The list cards does not contain number query.
The list cards is empty.
The list cards contains repeating numbers.
The number query occurs at more than one position in cards.
(can you think of any more variations?)
Edge Cases: It's likely that you didn't think of all of the above cases when you read the problem for the first time. Some of these (like the empty array or query not occurring in cards) are called edge cases, as they represent rare or extreme examples.

While edge cases may not occur frequently, your programs should be able to handle all edge cases, otherwise they may fail in unexpected ways. Let's create some more test cases for the variations listed above. We'll store all our test cases in an list for easier testing.

In [7]:
tests = []

# query occurs in the middle
tests.append(test)

tests.append({
    'input': {
        'cards': [13, 11, 10, 7, 4, 3, 1, 0],
        'query': 1
    },
    'output': 6
})

In [8]:
# query is the first element
tests.append({
    'input': {
        'cards': [4, 2, 1, -1],
        'query': 4
    },
    'output': 0
})

In [9]:
# query is the last element
tests.append({
    'input' : {
        'cards': [3, -1, -9, -127],
        'query': -127
    },
    'output': 3
})

In [10]:
# cards contain just one element (query)
tests.append({
    'input' : {
        'cards': [6],
        'query': 6
    },
    'output': 0
})

The problem statement does not specify what to do if the list cards does not contain the number query.

Read the problem statement again, carefully.
Look through the examples provided with the problem.
Ask the interviewer/platform for a clarification.
Make a reasonable assumption, state it and move forward.
We will assume that our function will return -1 in case cards does not contain query.

In [11]:
# cards does not contain query

tests.append({
    'input': {
        'cards': [9, 7, 5, 2, -9],
        'query': 4
    },
    'output': -1
})

In [12]:
# cards is empty

tests.append({
    'input':{
        'cards': [],
        'query': 7
    },
    'output': -1  
})

In [13]:
# numbers can repeat in cards
tests.append({
    'input': {
        'cards': [8, 8, 6, 6, 6, 6, 6, 3, 2, 2, 0, 0],
        'query': 3
    },
    'output': 7
})

In the case where query occurs multiple times in cards, we'll expect our function to return the first occurrence of query.

While it may also be acceptable for the function to return any position where query occurs within the list, it would be slightly more difficult to test the function, as the output is non-deterministic.

In [14]:
# query occurs multiple times
tests.append({
    'input': {
        'cards': [8, 8, 6, 6, 6, 6, 6, 6, 3, 2, 2, 2, 0, 0, 0],
        'query': 6
    },
    'output': 2
})

In [15]:
tests

[{'input': {'cards': [13, 11, 10, 7, 4, 3, 1, 0], 'query': 7}, 'output': 3},
 {'input': {'cards': [13, 11, 10, 7, 4, 3, 1, 0], 'query': 1}, 'output': 6},
 {'input': {'cards': [4, 2, 1, -1], 'query': 4}, 'output': 0},
 {'input': {'cards': [3, -1, -9, -127], 'query': -127}, 'output': 3},
 {'input': {'cards': [6], 'query': 6}, 'output': 0},
 {'input': {'cards': [9, 7, 5, 2, -9], 'query': 4}, 'output': -1},
 {'input': {'cards': [], 'query': 7}, 'output': -1},
 {'input': {'cards': [8, 8, 6, 6, 6, 6, 6, 3, 2, 2, 0, 0], 'query': 3},
  'output': 7},
 {'input': {'cards': [8, 8, 6, 6, 6, 6, 6, 6, 3, 2, 2, 2, 0, 0, 0],
   'query': 6},
  'output': 2}]

Our first goal should always be to come up with a correct solution to the problem, which may necessarily be the most efficient solution. The simplest or most obvious solution to a problem, which generally involves checking all possible answers is called the brute force solution.

In this problem, coming up with a correct solution is quite easy: Bob can simply turn over cards in order one by one, till he find a card with the given number on it. Here's how we might implement it:

Create a variable position with the value 0.
Check whether the number at index position in card equals query.
If it does, position is the answer and can be returned from the function
If not, increment the value of position by 1, and repeat steps 2 to 5 till we reach the last position.
If the number was not found, return -1.

This particular algorithm is called linear search, since it involves searching through a list in a linear fashion i.e. element after element.

In [16]:
def locate_card(cards, query):
    position = 0 # create a variable position with value 0

    while True:
        # check if the element at the current position match the query
        if cards[position] == query:
            return position
        
        position += 1 # increment the position

        # check if we have reached the end of the array

        if position == len(cards):
            return -1 # number not found


In [17]:
test

{'input': {'cards': [13, 11, 10, 7, 4, 3, 1, 0], 'query': 7}, 'output': 3}

In [18]:
result = locate_card(**test['input'])
print(result)

3


In [19]:
result == output

True

In [20]:
for test in tests:
    print(locate_card(**test['input']) == test['output'])

True
True
True
True
True
True


IndexError: list index out of range

Let's solve this problem now. I will use some print statements to have some visibility in the locate_card function.

In [21]:
def locate_card(cards, query):
    position = 0

    print('cards:', cards)
    print('query:', query)

    while True:
        print('position:', position)

        if cards[position] == query:
            return position
        position += 1

        if position == len(cards):
            return -1

In [22]:
cards6 = tests[6]['input']['cards']
print(cards6)

[]


In [23]:
query6 = tests[6]['input']['query']
print(query6)

7


In [24]:
locate_card(cards6, query6)

cards: []
query: 7
position: 0


IndexError: list index out of range

Clearly, since cards is empty, it's not possible to access the element at index 0. To fix this, we can check whether we've reached the end of the array before trying to access an element from it. In fact, this can be terminating condition for the while loop itself.

In [25]:
def locate_card(cards, query):
    position = 0

    while position < len(cards):
        if cards[position] == query:
            return position
        position += 1

    return -1

In [26]:
tests[6]

{'input': {'cards': [], 'query': 7}, 'output': -1}

In [27]:
   locate_card(cards6, query6)  # prevoius problem is solved

-1

 **Analyze the algorithm's complexity and identify inefficiencies, if any.**

Recall this statement from original question: "Alice challenges Bob to pick out the card containing a given number by turning over as few cards as possible." We restated this requirement as: "Minimize the number of times we access elements from the list cards"


Before we can minimize the number, we need a way to measure it. Since we access a list element once in every iteration, for a list of size N we access the elements from the list up to N times. Thus, Bob may need to overturn up to N cards in the worst case, to find the required card.

Suppose he is only allowed to overturn 1 card per minute, it may take him 30 minutes to find the required card if 30 cards are laid out on the table. Is this the best he can do? Is a way for Bob to arrive at the answer by turning over just 5 cards, instead of 30?

The field of study concerned with finding the amount of time, space or other resources required to complete the execution of computer programs is called the analysis of algorithms. And the process of figuring out the best algorithm to solve a given problem is called algorithm design and optimization.

**Complexity and Big O Notation**


Complexity of an algorithm is a measure of the amount of time and/or space required by an algorithm for an input of a given size e.g. N. Unless otherwise stated, the term complexity always refers to the worst-case complexity (i.e. the highest possible time/space taken by the program/algorithm to process an input).

In the case of linear search:

The time complexity of the algorithm is cN for some fixed constant c that depends on the number of operations we perform in each iteration and the time taken to execute a statement. Time complexity is sometimes also called the running time of the algorithm.

The space complexity is some constant c' (independent of N), since we just need a single variable position to iterate through the array, and it occupies a constant space in the computer's memory (RAM).

Big O Notation: Worst-case complexity is often expressed using the Big O notation. In the Big O, we drop fixed constants and lower powers of variables to capture the trend of relationship between the size of the input and the complexity of the algorithm i.e. if the complexity of the algorithm is cN^3 + dN^2 + eN + f, in the Big O notation it is expressed as O(N^3)

Thus, the time complexity of linear search is O(N) and its space complexity is O(1).



At the moment, we're simply going over cards one by one, and not even utilizing the face that they're sorted. This is called a brute force approach.

It would be great if Bob could somehow guess the card at the first attempt, but with all the cards turned over it's simply impossible to guess the right card.


The next best idea would be to pick a random card, and use the fact that the list is sorted, to determine whether the target card lies to the left or right of it. In fact, if we pick the middle card, we can reduce the number of additional cards to be tested to half the size of the list. Then, we can simply repeat the process with each half. This technique is called binary search. 



**Here's how binary search can be applied to our problem:**

Find the middle element of the list.
If it matches queried number, return the middle position as the answer.
If it is less than the queried number, then search the first half of the list
If it is greater than the queried number, then search the second half of the list
If no more elements remain, return -1.

In [28]:
## binary search

def locate_card(cards, query):
    lo, hi = 0, len(cards)-1

    while lo<=hi:
        mid  =  (lo + hi) // 2
        mid_number = cards[mid]

        print("lo:", lo, 
        "hi:", hi, 
        "mid:", mid, 
        "mid_number:", mid_number)

        if mid_number == query:
            print("Actual Output:", mid)
            return ""
        elif mid_number < query:
            hi = mid - 1
        elif mid_number > query:
            lo =  mid + 1
    return -1



In [29]:
for test in tests:
    #print(test['input']['query'])
    print(test['input']['cards'],"\nExpected Output:",test['output'])
    print(locate_card(**test['input']))

[13, 11, 10, 7, 4, 3, 1, 0] 
Expected Output: 3
lo: 0 hi: 7 mid: 3 mid_number: 7
Actual Output: 3

[13, 11, 10, 7, 4, 3, 1, 0] 
Expected Output: 6
lo: 0 hi: 7 mid: 3 mid_number: 7
lo: 4 hi: 7 mid: 5 mid_number: 3
lo: 6 hi: 7 mid: 6 mid_number: 1
Actual Output: 6

[4, 2, 1, -1] 
Expected Output: 0
lo: 0 hi: 3 mid: 1 mid_number: 2
lo: 0 hi: 0 mid: 0 mid_number: 4
Actual Output: 0

[3, -1, -9, -127] 
Expected Output: 3
lo: 0 hi: 3 mid: 1 mid_number: -1
lo: 2 hi: 3 mid: 2 mid_number: -9
lo: 3 hi: 3 mid: 3 mid_number: -127
Actual Output: 3

[6] 
Expected Output: 0
lo: 0 hi: 0 mid: 0 mid_number: 6
Actual Output: 0

[9, 7, 5, 2, -9] 
Expected Output: -1
lo: 0 hi: 4 mid: 2 mid_number: 5
lo: 3 hi: 4 mid: 3 mid_number: 2
-1
[] 
Expected Output: -1
-1
[8, 8, 6, 6, 6, 6, 6, 3, 2, 2, 0, 0] 
Expected Output: 7
lo: 0 hi: 11 mid: 5 mid_number: 6
lo: 6 hi: 11 mid: 8 mid_number: 2
lo: 6 hi: 7 mid: 6 mid_number: 6
lo: 7 hi: 7 mid: 7 mid_number: 3
Actual Output: 7

[8, 8, 6, 6, 6, 6, 6, 6, 3, 2, 2, 2, 0, 

[8, 8, 6, 6, 6, 6, 6, 6, 3, 2, 2, 2, 0, 0, 0] 
Actual Output: 2
lo: 0 hi: 14 mid: 7 mid_number: 6
Output we got is 7. So this test case has failed here. Seems like we did locate a 6 in the array, it's just that it wasn't the first 6. We got the mid as 7 and the 7th element in the card is 6. As you can guess, this is because in binary search, we don't go over indices in a linear order. 

So how do we fix it?

When we find that cards[mid] is equal to query, we need to check whether it is the first occurrence of query in the list i.e the number that comes before it.

[8, 8, 6, 6, 6, 6, 6, 6, 3, 2, 2, 2, 0, 0, 0]

To make it easier, we'll define a helper function called test_location, which will take the list cards, the query and mid as inputs.

In [34]:
def test_location(cards, query, mid):
    mid_number = cards[mid]
    print("mid", mid, ",mid number", mid_number)

    if mid_number == query:
        if mid - 1 >= 0 and cards[mid-1] == query:
            return 'left'
        else:
            return 'found'
    elif mid_number < query:
        return 'left'
    else:
        return 'right'


def locate_card(cards, query):
    lo, hi = 0, len(cards) - 1

    while lo <= hi:
        print("lo:", lo, ",hi:", hi)
        mid = (lo + hi) // 2
        result = test_location(cards, query, mid)

        if result == 'found':
            print("Actual Output:", mid)
            return " "
        elif result == 'left':
            hi = mid - 1
        elif result == 'right':
            lo = mid + 1
        
    return -1


In [35]:
for test in tests:
    print(test['input']['cards'],"\nExpected Output:",test['output'])
    print(locate_card(**test['input']))

[13, 11, 10, 7, 4, 3, 1, 0] 
Expected Output: 3
lo: 0 ,hi: 7
mid 3 ,mid number 7
Actual Output: 3
 
[13, 11, 10, 7, 4, 3, 1, 0] 
Expected Output: 6
lo: 0 ,hi: 7
mid 3 ,mid number 7
lo: 4 ,hi: 7
mid 5 ,mid number 3
lo: 6 ,hi: 7
mid 6 ,mid number 1
Actual Output: 6
 
[4, 2, 1, -1] 
Expected Output: 0
lo: 0 ,hi: 3
mid 1 ,mid number 2
lo: 0 ,hi: 0
mid 0 ,mid number 4
Actual Output: 0
 
[3, -1, -9, -127] 
Expected Output: 3
lo: 0 ,hi: 3
mid 1 ,mid number -1
lo: 2 ,hi: 3
mid 2 ,mid number -9
lo: 3 ,hi: 3
mid 3 ,mid number -127
Actual Output: 3
 
[6] 
Expected Output: 0
lo: 0 ,hi: 0
mid 0 ,mid number 6
Actual Output: 0
 
[9, 7, 5, 2, -9] 
Expected Output: -1
lo: 0 ,hi: 4
mid 2 ,mid number 5
lo: 3 ,hi: 4
mid 3 ,mid number 2
-1
[] 
Expected Output: -1
-1
[8, 8, 6, 6, 6, 6, 6, 3, 2, 2, 0, 0] 
Expected Output: 7
lo: 0 ,hi: 11
mid 5 ,mid number 6
lo: 6 ,hi: 11
mid 8 ,mid number 2
lo: 6 ,hi: 7
mid 6 ,mid number 6
lo: 7 ,hi: 7
mid 7 ,mid number 3
Actual Output: 7
 
[8, 8, 6, 6, 6, 6, 6, 6, 3, 2, 2, 

Once again, let's try to count the number of iterations in the algorithm. If we start out with an array of N elements, then each time the size of the array reduces to half for the next iteration, until we are left with just 1 element.

Initial length - N

Iteration 1 - N/2

Iteration 2 - N/4 i.e. N/2^2

Iteration 3 - N/8 i.e. N/2^3

...

Iteration k - N/2^k

Since the final length of the array is 1, we can find the

N/2^k = 1

Rearranging the terms, we get

N = 2^k

Taking the logarithm

k = log N

Where log refers to log to the base 2. Therefore, our algorithm has the time complexity O(log N). This fact is often stated as: binary search runs in logarithmic time. You can verify that the space complexity of binary search is O(1).

**Binary Search vs. Linear Search**

Furthermore, as the size of the input grows larger, the difference only gets bigger. For a list 10 times, the size, linear search would run for 10 times longer, whereas binary search would only require 3 additional operations! (can you verify this?) That's the real difference between the complexities O(N) and O(log N).

Another way to look at it is that binary search runs c * N / log N times faster than linear search, for some fixed constant c. Since log N grows very slowly compared to N, the difference gets larger with the size of the input.

**Generic Binary Search**

Here is the general strategy behind binary search, which is applicable to a variety of problems:

Come up with a condition to determine whether the answer lies before, after or at a given position

Retrieve the midpoint and the middle element of the list.

If it is the answer, return the middle position as the answer.

If answer lies before it, repeat the search with the first half of the list

If the answer lies after it, repeat the search with the second half of the list.

Here is the generic algorithm for binary search, implemented in Python:

In [36]:
def binary_search(lo, hi, condition):
    '''TODO - describe the function in this documentation'''
    while lo <= hi:
        mid = (lo + hi) // 2
        result = condition(mid)

        if result == 'found':
            return mid
        elif result == 'right':
            lo = mid + 1
        elif result == 'left':
            hi = mid - 1

    return -1

The worst-case complexity or running time of binary search is O(log N), provided the complexity of the condition used to determine whether the answer lies before, after or at a given position is O(1).

Note that binary_search accepts a function condition as an argument. Python allows passing functions as arguments to other functions, unlike C++ and Java.

We can now rewrite the locate_card function more succinctly using the binary_search function.

In [41]:
def locate_card(cards, query): # condition() have access to cards and query as it lies inside this function

    def condition(mid): # function closure: writing function inside a function
        if cards[mid] == query:
            if mid > 0 and cards[mid-1] == query:
                return 'left'
            else:
                return 'found'
        elif cards[mid] < query:
            return 'left'
        else:
            return 'right'

    return binary_search(0, len(cards) - 1, condition)

In [42]:
for test in tests:
    print(test['input']['cards'],"\nExpected Output:",test['output'])
    print(locate_card(**test['input']))


[13, 11, 10, 7, 4, 3, 1, 0] 
Expected Output: 3
3
[13, 11, 10, 7, 4, 3, 1, 0] 
Expected Output: 6
6
[4, 2, 1, -1] 
Expected Output: 0
0
[3, -1, -9, -127] 
Expected Output: 3
3
[6] 
Expected Output: 0
0
[9, 7, 5, 2, -9] 
Expected Output: -1
-1
[] 
Expected Output: -1
-1
[8, 8, 6, 6, 6, 6, 6, 3, 2, 2, 0, 0] 
Expected Output: 7
7
[8, 8, 6, 6, 6, 6, 6, 6, 3, 2, 2, 2, 0, 0, 0] 
Expected Output: 2
2


**Question:**
Given an array of integer nums sorted in ascending order, find the starting and ending position of a given number.

This differs from the problem in only two significant ways:

1. The numbers are sorted in an increasing order.

2. We are looking for both start index and the end index.

Here's the full code for solving the question, obtained by minor modifications to our previous function.

In [44]:
def first_position(nums, target):
    def condition(mid):
        if nums[mid] == target:
            if mid > 0 and nums[mid -1] == target:
                return 'left'
            return 'found'

        elif nums[mid] < target:
            return 'right'
        else:
            return 'left'
    return binary_search(0, len(nums) - 1, condition)

def last_position(nums, target):
    def condition(mid):
        if nums[mid] == target:
            if mid < len(nums) - 1 and nums[mid + 1] == target:
                return 'right'
            return 'found'

        elif nums[mid] < target:
            return 'right'

        else:
            return 'left'

    return binary_search(0, len(nums) - 1, condition)

def first_and_last_position(nums, target):
    return first_position(nums, target), last_position(nums, target)
        
        
